In [ ]:
#online_train

handle_channel->rasa_core.channel.console.py
rasa_core->policies.py->ensemble.py
根据最大置信度预测
1.
rasa_core.agent.py
def train_online(self,
                     training_trackers,  # type: List[DialogueStateTracker]
                     input_channel=None,  # type: Optional[InputChannel]
                     max_visual_history=3,  # type: int
                     **kwargs  # type: **Any
                     ):
        # type: (...) -> None
        from rasa_core.policies.online_trainer import OnlinePolicyEnsemble
        """Train a policy ensemble in online learning mode."""

        if not self.interpreter:
            raise ValueError(
                    "When using online learning, you need to specify "
                    "an interpreter for the agent to use.")

        # TODO: DEPRECATED - remove in version 0.10
        if isinstance(training_trackers, string_types):
            # the user most likely passed in a file name to load training
            # data from
            logger.warning("Passing a file name to `agent.train_online(...)` "
                           "is deprecated. Rather load the data with "
                           "`data = agent.load_data(file_name)` and pass it "
                           "to `agent.train_online(data)`.")
            training_trackers = self.load_data(training_trackers)#story.md
     
        logger.debug("Agent online trainer got kwargs: {}".format(kwargs))
        check_domain_sanity(self.domain)
        #对于每一个策略训练,根据数据集和领域
        self.policy_ensemble.train(training_trackers, self.domain, **kwargs)

        ensemble = OnlinePolicyEnsemble(self.policy_ensemble,
                                        training_trackers,
                                        max_visual_history)

        ensemble.run_online_training(self.domain, self.interpreter,
                                     input_channel)

training_trackers->  training_data(story.md)
self.policy_ensemble.train(training_trackers, self.domain, **kwargs)

--------------------------------------------------------------
agent.py
def load_data(self,
                  resource_name,  # type: Text
                  remove_duplicates=True,  # type: bool
                  unique_last_num_states=None,  # type: Optional[int]
                  augmentation_factor=20,  # type: int
                  max_number_of_trackers=None,  # deprecated
                  tracker_limit=None,  # type: Optional[int]
                  use_story_concatenation=True,  # type: bool
                  debug_plots=False  # type: bool
                  ):
        # type: (...) -> List[DialogueStateTracker]
        """Load training data from a resource."""

        # find maximum max_history
        # and if all featurizers are MaxHistoryTrackerFeaturizer
        max_max_history = 0
        all_max_history_featurizers = True
        for policy in self.policy_ensemble.policies:
            if hasattr(policy.featurizer, 'max_history'):
                max_max_history = max(policy.featurizer.max_history,
                                      max_max_history)
            elif policy.featurizer is not None:
                all_max_history_featurizers = False

        if unique_last_num_states is None:
            # for speed up of data generation
            # automatically detect unique_last_num_states
            # if it was not set and
            # if all featurizers are MaxHistoryTrackerFeaturizer
            if all_max_history_featurizers:
                unique_last_num_states = max_max_history
        elif unique_last_num_states < max_max_history:
            # possibility of data loss
            logger.warning("unique_last_num_states={} but "
                           "maximum max_history={}."
                           "Possibility of data loss. "
                           "It is recommended to set "
                           "unique_last_num_states to "
                           "at least maximum max_history."
                           "".format(unique_last_num_states, max_max_history))

        return training.load_data(resource_name, self.domain,
                                  remove_duplicates, unique_last_num_states,
                                  augmentation_factor, max_number_of_trackers,
                                  tracker_limit, use_story_concatenation,
                                  debug_plots)
----------------------------------------------------------------------------
ensemble.py->self.policy_ensemble.train
def train(self, training_trackers, domain, **kwargs):
        # type: (List[DialogueStateTracker], Domain, **Any) -> None
        if training_trackers:
            for policy in self.policies:
                policy.train(training_trackers, domain, **kwargs)
            self.training_trackers = training_trackers
        else:
            logger.info("Skipped training, because there are no "
                        "training samples.")
-------------------------------------------------------------------------
KerasPolicy()
def train(self,
              training_trackers,  # type: List[DialogueStateTracker]
              domain,  # type: Domain
              **kwargs  # type: **Any
              ):
        # type: (...) -> Dict[Text: Any]

        if kwargs.get('rnn_size') is not None:
            logger.debug("Parameter `rnn_size` is updated with {}"
                         "".format(kwargs.get('rnn_size')))
            self.rnn_size = kwargs.get('rnn_size')

        training_data = self.featurize_for_training(training_trackers,
                                                    domain,
                                                    **kwargs)
---------------------------------------------------------------------
policy.py 
def featurize_for_training(
            self,
            training_trackers,  # type: List[DialogueStateTracker]
            domain,  # type: Domain
            **kwargs  # type: **Any
    ):
        # type: (...) -> DialogueTrainingData
        """Transform training trackers into a vector representation.
        The trackers, consisting of multiple turns, will be transformed
        into a float vector which can be used by a ML model."""

        training_data = self.featurizer.featurize_trackers(training_trackers,
                                                           domain)

        max_training_samples = kwargs.get('max_training_samples')
        if max_training_samples is not None:
            logger.debug("Limit training data to {} training samples."
                         "".format(max_training_samples))
            training_data.limit_training_data_to(max_training_samples)

        return training_data
featurizers.py
MaxHistoryTrackerFeaturizer
---------------------------------

---------------------------------



----------------------------------------------------------------------
        shuffled_X, shuffled_y = training_data.shuffled_X_y()

        if self.model is None:
            self.model = self.model_architecture(shuffled_X.shape[1:],
                                                 shuffled_y.shape[1:])

        validation_split = kwargs.get("validation_split", 0.0)
        logger.info("Fitting model with {} total samples and a validation "
                    "split of {}".format(training_data.num_examples(),
                                         validation_split))
        # filter out kwargs that cannot be passed to fit
        params = self._get_valid_params(self.model.fit, **kwargs)

        self.model.fit(shuffled_X, shuffled_y, **params)
        # the default parameter for epochs in keras fit is 1
        self.current_epoch = kwargs.get("epochs", 1)
        logger.info("Done fitting keras policy model")